In [1]:
from scipy.optimize import differential_evolution
import tensorflow as tf
import configs
import models

In [2]:
run_config = configs.get_model_config(
    dataset_name="cifar10", num_classes=10, image_size=32, path="."
)
dataset_config = run_config["dataset_config"]

In [3]:
batch_size = run_config["batch_size"]
(_, _), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_ds = test_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

2022-01-02 15:11:51.132633: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
model_ckpt_paths = tf.io.gfile.glob("*-cifar10")
sorted(model_ckpt_paths)

['./220102-090159-cifar10', './220102-090412-cifar10']

In [5]:
model_one = models.get_shallow_cnn(run_config)
model_one.load_weights(tf.train.latest_checkpoint(model_ckpt_paths[0]))

model_two = models.get_shallow_cnn(run_config)
model_two.load_weights(tf.train.latest_checkpoint(model_ckpt_paths[1]))

print("")

In [6]:
ensembled_model = models.get_shallow_cnn(run_config)
ensembled_model.compile(
    loss="sparse_categorical_crossentropy", metrics=["accuracy"], optimizer="adam"
)
ema = 0.01

for i in range(len(ensembled_model.layers)):
    if hasattr(ensembled_model.layers[i], "kernel") and (
        hasattr(ensembled_model.layers[i], "bias")
    ):
        ensembled_weights = (
            model_one.layers[i].kernel * ema + (1 - ema) * model_two.layers[i].kernel
        )

        ensembled_bias = (
            model_one.layers[i].bias * ema + (1 - ema) * model_two.layers[i].bias
        )

        ensembled_model.layers[i].kernel.assign(tf.Variable(ensembled_weights))
        ensembled_model.layers[i].bias.assign(tf.Variable(ensembled_bias))

In [7]:
_, accuracy = ensembled_model.evaluate(test_ds)
print(f"Test accuracy: {round(accuracy * 100, 2)}%")

20/20 [==============================] - 1s 30ms/step - loss: 1.0434 - accuracy: 0.6333
Test accuracy: 63.33%


In [8]:
def evaluate_model(ema_factor, members, test_ds):
    ensembled_model = models.get_shallow_cnn(run_config)
    ensembled_model.compile(
        loss="sparse_categorical_crossentropy", metrics=["accuracy"], optimizer="adam"
    )

    for i in range(len(ensembled_model.layers)):
        if hasattr(ensembled_model.layers[i], "kernel") and (
            hasattr(ensembled_model.layers[i], "bias")
        ):
            ensembled_weights = (
                members[0].layers[i].kernel * ema
                + (1 - ema) * members[1].layers[i].kernel
            )

            ensembled_bias = (
                members[0].layers[i].bias * ema_factor
                + (1 - ema_factor) * members[1].layers[i].bias
            )

            ensembled_model.layers[i].kernel.assign(tf.Variable(ensembled_weights))
            ensembled_model.layers[i].bias.assign(tf.Variable(ensembled_bias))

    _, accuracy = ensembled_model.evaluate(test_ds, verbose=0)
    return accuracy


def loss_evolution(ema_factor, members, test_ds):
    return 1.0 - evaluate_model(ema_factor, members, test_ds)

In [9]:
# Reference: https://machinelearningmastery.com/weighted-average-ensemble-for-deep-learning-neural-networks/
members = [model_one, model_two]

search_arg = (members, test_ds)
result = differential_evolution(
    loss_evolution, bounds=[(0, 1)], args=search_arg, maxiter=1000, tol=1e-7,
)

In [11]:
print(f"EMA factor found using differential evolution: {result['x']}.")
accuracy = evaluate_model(result["x"], members, test_ds)
print(f"Test accuracy: {round(accuracy * 100, 2)}%")

EMA factor found using differential evolution: [0.02048232].
Test accuracy: 63.5%


In [34]:
ensembled_model = models.get_shallow_cnn(run_config)
ensembled_model.compile(
    loss="sparse_categorical_crossentropy", metrics=["accuracy"], optimizer="adam"
)

_, accuracy = ensembled_model.evaluate(test_ds)
print(f"Test accuracy: {round(accuracy * 100, 2)}%")

20/20 [==============================] - 1s 30ms/step - loss: 2.3025 - accuracy: 0.1040
Test accuracy: 10.4%


In [13]:
for model in members:
    model.compile(
        loss="sparse_categorical_crossentropy", metrics=["accuracy"], optimizer="adam"
    )

    _, accuracy = model.evaluate(test_ds)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

20/20 [==============================] - 1s 30ms/step - loss: 1.0428 - accuracy: 0.6323
Test accuracy: 63.23%
20/20 [==============================] - 1s 30ms/step - loss: 1.0436 - accuracy: 0.6342
Test accuracy: 63.42%
